In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, message="Resolving lazy import graphs")

import time
import random
import netdecom as nd
from sage.graphs.all import *
import pandas as pd
import importlib.util 
import sys 
import os 
so_path = "./libdecom_h.so"
module_name = "decom_h" 
spec = importlib.util.spec_from_file_location(module_name, so_path) 
decom_h = importlib.util.module_from_spec(spec) 
sys.modules[module_name] = decom_h 
spec.loader.exec_module(decom_h) 

<module 'decom_h' from '/home/jovyan/work/CMSA_IPA_SAHR/libdecom_h.so'>


In [ ]:
repeat = 100  

results = []

for n in [2500,5000,7500,10000]:
    for p in [0.1, 0.01, 0.005, 0.001]:
        total_cmsa_time, total_ipa_time = 0, 0  

        for i in range(repeat):
            G_ig = nd.generator_connected_ug(n, p, class_type="ig")
            r = random.sample(range(n), 20)


            start_time = time.time()
            result1 =  decom_h.find_convex_hull(G_ig, r,  method="cmsa")
            cmsa_time = time.time() - start_time
            total_cmsa_time += cmsa_time  


            start_time = time.time()
            result2 =  decom_h.find_convex_hull(G_ig, r,  method="ipa")
            ipa_time = time.time() - start_time
            total_ipa_time += ipa_time  


            if result1 != result2:
                print("Error")
                break

            results.append({
                'n': n,
                'p': p,
                'iteration': i + 1,
                'cmsa_time': cmsa_time,
                'ipa_time': ipa_time
            })

        avg_cmsa_time = total_cmsa_time / repeat
        avg_ipa_time = total_ipa_time / repeat
        print(f"n={n}, p={p}, CMSA average running time={avg_cmsa_time:.4f}, IPA average running time={avg_ipa_time:.4f}")


df = pd.DataFrame(results)
df.to_csv('experiment_results.csv', index=False)

